In [8]:
# Import libraries

import pandas as pd
import numpy as np

import matplotlib
import plotly.express as px

import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

import geopandas as gpd

# Reading data
Different types of files can contain geographic data : geojson, shapefiles (.shp),... but also CSV or Excel.

In [9]:
df_data_nationales = pd.read_pickle('../data/Pickles/df_data_nationales.pkl')
df_eolien_norm = pd.read_pickle('../data/Pickles/df_eolien_norm.pkl')
df_solaire_norm = pd.read_pickle('../data/Pickles/df_solaire_norm.pkl')
df_temperatures = pd.read_pickle('../data/Pickles/df_temperatures.pkl')

### 1. Foisonnement de la production renouvelable en France :
idée 1 : Par région : Barplot de la production anuelle par région, stacked (solaire éolien)\
idée 2 : Évoluation (lineplot) au cours d'une année\
    - faire des courbes différente par région ou type d'énergie produite.

### 2. Thermo-sensibilité de la consommation :
idée 1 : Tracer deux lineplot évolution de la température dans le temps et évolution de la consommation dans le temps.
        (moyenné par journée)

idée optionelle : effet de la canicule sur la consommation

### 3. Mix énergétique en France :
Stacked plot

In [10]:
#input
# df_data_nationales = pd.read_pickle('../data/Pickles/df_data_nationales.pkl')
#Data Processing
#sampling and choosing features
df_energy_monthly = df_data_nationales.resample('ME').sum()
# df_energy_monthly = df_energy.set_index('DateTime').resample('ME').sum()
col_prod = ['Nucléaire','Hydraulique','Gaz','Eolien','Charbon','Bioénergies','Solaire','Fioul'].__reversed__()
df_energy_monthly[col_prod]
#computing proportions
df_energy_monthly_total = df_energy_monthly[col_prod].sum(axis=1)
df_prop = pd.DataFrame(index = df_energy_monthly.index.copy())
for c in col_prod:
    df_prop[c] = df_energy_monthly[c]/df_energy_monthly_total*100

#ploting
import plotly.graph_objects as go

fig = go.Figure()
# for prod_src in col_prod.__reversed__():
for prod_src in col_prod:
    fig.add_trace(go.Scatter(
        x=df_energy_monthly.index,
        y=df_prop[prod_src],
        fill='tonexty',
        name=prod_src,
        stackgroup='one'
    ))

fig.update_layout(title='Proportions du mix Énergétique au cours du temps', xaxis_title='Temps', yaxis_title='proportion (en %)')
fig.show()


In [11]:
#input
# df_data_nationales = pd.read_pickle('../data/Pickles/df_data_nationales.pkl')
#Data Processing
#sampling and choosing features
df_energy_monthly = df_data_nationales.resample('ME').mean()
# df_energy_monthly = df_energy.set_index('DateTime').resample('ME').sum()
col_prod = ['Nucléaire','Hydraulique','Gaz','Eolien','Charbon','Bioénergies','Solaire','Fioul']
df_energy_monthly[col_prod]
#ploting
import plotly.graph_objects as go

fig = go.Figure()
# for prod_src in col_prod.__reversed__():
for prod_src in col_prod:
    fig.add_trace(go.Scatter(
        x=df_energy_monthly.index,
        y=df_energy_monthly[prod_src],
        fill='tonexty',
        name=prod_src,
        stackgroup='one'
    ))

fig.update_layout(title='Évolution temporelle de la moyenne mensuelle du mix de production Énergétique au cours du temps', xaxis_title='Temps', yaxis_title='MW (en %)')
fig.show()

In [ ]:
import statsmodels.api as sm
##################
# df_energy_monthly = df_energy.set_index('DateTime').resample('ME').sum()
selected_options_1 = ['Nucléaire']
option_2 = 'Rien'
col_group = ['Groupe_1','Groupe_2']
col_g1 = list(selected_options_1)
if option_2 == 'Rien':
    col_group = ['Groupe_1']
    title=f"Évolution de la production groupée,<br>{col_group[0]} : {', '.join(col_g1)}"
else :
    col_g2 = list(set(col_prod) - set(col_g1))
    col_group = ['Groupe_1','Groupe_2']
    title=f"Proportions du mix Énergétique au cours du temps,<br>{col_group[0]} : {', '.join(col_g1)},<br>{col_group[1]} : {', '.join(col_g2)}"
  
df_energy_group = pd.DataFrame()
if option_2 == 'Rien':
    df_energy_group[col_group[0]] = df_energy_monthly[col_g1].sum(axis=1)
else : 
#computing proportions
    df_energy_group[col_group[0]] = df_energy_monthly[col_g1].sum(axis=1)/  df_energy_monthly_total *100
    df_energy_group[col_group[1]] = df_energy_monthly[col_g2].sum(axis=1)/  df_energy_monthly_total *100
########################
df_months = pd.DataFrame(df_energy_group.index)
df_months = df_months.reset_index()
df_months['y'] = df_energy_group[col_group[0]]
Y = df_months['y']
X = df_months['index']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params
df_months = df_months.reset_index()
# df_months['y'] = 
df_energy_group

,Groupe_1
DateTime,
2012-01-31,57495.315188
2012-02-29,55535.587644
2012-03-31,50482.514113
2012-04-30,45939.453472
2012-05-31,39021.690188
...,...
2018-08-31,36670.907258
2018-09-30,41779.852083
2018-10-31,42414.565860


In [52]:
df_pred = df_energy_group.reset_index().reset_index()
Y = df_pred[col_group[0]]
X = df_pred['index']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
a,b = results.params[1],results.params[0]
df_pred['y_pred'] = a*df_pred['index']+b
df_pred

/tmp/ipykernel_10112/3110877298.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,index,DateTime,Groupe_1,y_pred
0,0,2012-01-31,57495.315188,47635.695871
1,1,2012-02-29,55535.587644,47586.347024
2,2,2012-03-31,50482.514113,47536.998177
3,3,2012-04-30,45939.453472,47487.649330
4,4,2012-05-31,39021.690188,47438.300483
...,...,...,...,...
79,79,2018-08-31,36670.907258,43737.136955
80,80,2018-09-30,41779.852083,43687.788108
81,81,2018-10-31,42414.565860,43638.439261
82,82,2018-11-30,45335.424306,43589.090414


In [18]:
import statsmodels.api as sm
import numpy as np
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params

const        10.603498
education     0.594859
dtype: float64